<a href="https://colab.research.google.com/github/Mayank-004/VC-Industry-Predictive-analysis/blob/main/VC_Target_MOIC_KMEANS_CLUSTERING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [431]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from sklearn.preprocessing import OrdinalEncoder


## Importing the dataset







In [432]:
url1 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VC_Exits.csv'
dataset1 = pd.read_csv(url1, error_bad_lines=False)

#url2 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VCExit2.csv'
#dataset2 = pd.read_csv(url2, error_bad_lines=False)

url3 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VCData2.csv'
dataset3 = pd.read_csv(url3, error_bad_lines=False)

dataset = pd.merge(dataset1,dataset3,left_on=["Company PBID"],right_on=["Company ID"])



dataset


,Company PBID,Exit Date,Exit Size (millions),Pre Value (millions),Post Value (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,Total Preferred Capital Raised ($M)*,% Preferred Capital Raised*,Total Preferred Capital Raised ($M) - Combination**,% Preferred Capital Raised - Combination**,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,State,Country,Deal ID,Company ID,Primary Industry Sector,Primary Industry Group,Primary Industry Code,All Industries,Verticals_y,Keywords,Current Financing Status,Current Business Status,Universe,CEO PBId,CEO Education,Deal No.,Deal ID.1,Announced Date,Deal Date,Deal Size,Deal Size Status,Pre-money Valuation,...,# Investors,# New Investors,# Follow-on Investors,Implied EV,Revenue,Revenue Growth since last debt deal,Gross Profit,Net Income,EBITDA,EBIT,Total Debt (from financials),Fiscal Year,Valuation/EBITDA,Valuation/EBIT,Valuation/Net Income,Valuation/Revenue,Valuation/Cash Flow,Deal Size/EBITDA,Deal Size/EBIT,Deal Size/Net Income,Deal Size/Revenue,Deal Size/Cash Flow,Debt/EBITDA,Debt/Equity,Implied EV/EBITDA,Implied EV/EBIT,Implied EV/Net Income,Implied EV/Revenue,Implied EV/Cash Flow,EBITDA Margin %,Current Employees,Native Currency of Deal,HQ Location,HQ Global Region,HQ Global Sub Region,Company City,Company State/Province,Company Post Code,Company Country,Year Founded
0,155748-70,11/15/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,NaN,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,Utah,United States,67911-49T,155748-70,Information Technology,Software,Other Software,"Business/Productivity Software, Other Software*","3D Printing, CloudTech & DevOps, TMT","additives manufacturing software, additives pr...",Formerly VC-backed,Generating Revenue,Venture Capital,131029-48P,"Texas A&M University, Ph.D. (Doctor of Philoso...",1,67911-49T,NaN,01-Jan-2015,0.33,Actual,NaN,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,US Dollars (USD),"Park City, UT",Americas,North America,Park City,Utah,84098,United States,2014.0
1,155748-70,11/15/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,NaN,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,Utah,United States,122433-04T,155748-70,Information Technology,Software,Other Software,"Business/Productivity Software, Other Software*","3D Printing, CloudTech & DevOps, TMT","additives manufacturing software, additives pr...",Formerly VC-backed,Generating Revenue,Venture Capital,131029-48P,"Texas A&M University, Ph.D. (Doctor of Philoso...",2,122433-04T,NaN,26-May-2015,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,US Dollars (USD),"Park City, UT",Americas,North America,Park City,Utah,84098,United States,2014.0
2,160695-55,1/13/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,$0.70,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,NaN,Israel,71323-57T,160695-55,Information Technology,Software,Software Development Applications,"Application Software, Software Development App...","CloudTech & DevOps, SaaS, TMT","a/b testing, application development, applicat...",Formerly VC-backed,Generating Revenue,Venture Capital,56786-14P,"BA (Bachelor of Arts), 2005, Business & Marketing",1,71323-57T,NaN,18-Mar-2016,0.70,Actual,NaN,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,US Dollars (USD),"Tel Aviv, Israel",Middle East,Middle East,Tel Aviv,NaN,NaN,Israel,2015.0
3,81857-08,3/18/19,$166.00,NaN,$166.00,Merger/Acquisition,Early Stage VC,$10.10,NaN,NaN,NaN,NaN,16.44x,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,Texas,United States,46035

### Filling missing values of Target column MOIC

In [433]:
# target column MOIC is derived by Exit Size divided by Total VC capital raised
dataset['Total VC Capital Raised ($M)***']= dataset['Total VC Capital Raised ($M)***'].str.replace("$","") # removing $ from Total VC capital
dataset['Total VC Capital Raised ($M)***']= dataset['Total VC Capital Raised ($M)***'].str.replace(",","") # removing "," from Total VC capital
dataset.loc[:,'Total VC Capital Raised ($M)***'] = dataset.loc[:,'Total VC Capital Raised ($M)***'].astype(float) # converting Total VC capital to float from string
dataset['Total VC Capital Raised ($M)***']
dataset['Exit Size (millions)']= dataset['Exit Size (millions)'].str.replace("$","") # removing $ from Exit Size
dataset['Exit Size (millions)']= dataset['Exit Size (millions)'].str.replace(",","") # removing "," from Exit Size
dataset.loc[:,'Exit Size (millions)'] = dataset.loc[:,'Exit Size (millions)'].astype(float) # converting Total VC capital to float from string

dataset['MOIC']= dataset['MOIC'].str.replace("x","") # removing $ from MOIC
dataset['MOIC']= dataset['MOIC'].str.replace(",","") # removing ',' from MOIC
dataset.loc[:,'MOIC'] = dataset.loc[:,'MOIC'].astype(float) # converting MOIC to float from string

#dataset=dataset.drop(columns=['Exit Size (millions)','Total VC Capital Raised ($M)***'])
# Drop duplicate columns  

dataset=dataset.drop(columns=['Deal ID','Company ID','Primary Industry Sector','Primary Industry Group','Primary Industry Code','All Industries','Verticals_y','HQ Location','HQ Global Region','HQ Global Sub Region','Company City',		'Company Country'])

# Drop Column intuitively which may not have significant impact
dataset=dataset.drop(columns=['Company PBID','Exit Date','Post Value (millions)','Native Currency of Deal','Keywords','Deal No.','Deal ID.1','Deal Date','Deal Class','Deal Status','CEO PBId'])


In [434]:
# check missing value in dataset
missing = dataset.isna().sum().sort_values(ascending = False)
percent_missing = ((missing / dataset.isnull().count()) * 100).sort_values(ascending = False)
missing_df = pd.concat([missing,percent_missing], axis = 1, keys = ['Total', 'Percent'],sort = False)
missing_df[missing_df['Total'] >= 1]

,Total,Percent
Deal Type 3,305,100.000000
Implied EV,305,100.000000
Debt Type 3,305,100.000000
Implied EV/Cash Flow,305,100.000000
Implied EV/Revenue,305,100.000000
...,...,...
Company Post Code,28,9.180328
Raised to Date,24,7.868852
Total VC Capital Raised ($M)***,23,7.540984
Year Founded,5,1.639344


Drop Column which has missing values more than 5%

---



In [435]:
missing_df = missing_df.reset_index()
over30 = list(missing_df.loc[missing_df['Percent'] >= 5,'index'])
dataset = dataset.drop(over30, axis = 1)

In [436]:
dataset

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Year Founded
0,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Grant,Generating Revenue,Corporation,2014.0
1,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,2014.0
2,Merger/Acquisition,Early Stage VC,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,Israel,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Product In Beta Test,Venture Capital-Backed,2015.0
3,Merger/Acquisition,Early Stage VC,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,2013.0
4,Merger/Acquisition,Seed Round,Information Technology,Software,Application Software,"Mobile, Artificial Intelligence & Machine Lear...",Minsk,Belarus,Formerly VC-backed,Startup,Venture Capital,Seed Round,Startup,Venture Capital-Backed,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2001.0
301,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2001.0
302,IPO,Corporate,Information Technology,Computer Hardware,"Computers, Parts and Peripherals","Manufacturing, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Profitable,Venture Capital-Backed,2008.0
303,IPO,Series D,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2011.0


In [437]:
# calculating total working duration of company


from datetime import date 
todays_date = date.today() 

ts =todays_date.year

dataset['Total duration for Company'] = ts - dataset['Year Founded']

dataset=dataset.drop(columns=['Year Founded'])

dataset['Total duration for Company'] 

0       7.0
1       7.0
2       6.0
3       8.0
4       5.0
       ... 
300    20.0
301    20.0
302    13.0
303    10.0
304    10.0
Name: Total duration for Company, Length: 305, dtype: float64

In [438]:
dataset

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
0,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Grant,Generating Revenue,Corporation,7.0
1,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,7.0
2,Merger/Acquisition,Early Stage VC,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,Israel,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Product In Beta Test,Venture Capital-Backed,6.0
3,Merger/Acquisition,Early Stage VC,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
4,Merger/Acquisition,Seed Round,Information Technology,Software,Application Software,"Mobile, Artificial Intelligence & Machine Lear...",Minsk,Belarus,Formerly VC-backed,Startup,Venture Capital,Seed Round,Startup,Venture Capital-Backed,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,20.0
301,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,20.0
302,IPO,Corporate,Information Technology,Computer Hardware,"Computers, Parts and Peripherals","Manufacturing, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Profitable,Venture Capital-Backed,13.0
303,IPO,Series D,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,10.0


## Filling Missing Values

In [439]:
# check missing value in df
missing = dataset.isna().sum().sort_values(ascending = False)
percent_missing = ((missing / dataset.isnull().count()) * 100).sort_values(ascending = False)
missing_df = pd.concat([missing,percent_missing], axis = 1, keys = ['Total', 'Percent'],sort = False)
missing_df[missing_df['Total'] >= 1]

,Total,Percent
Total duration for Company,5,1.639344
Financing Status,1,0.327869


In [440]:
# filling missing values in all other columns with mean 

dataset = dataset.fillna((dataset.mean()))

dataset.dropna(subset = ["Financing Status"], inplace=True)
#dataset.apply(lambda x: x.fillna(x.mean()),axis=0)

## Encoding categorical data

Label Encoding of Categorical Variable

In [441]:
#Encoder
def label_encoder(dataset):

    #Takes in dataset, returns label encoded dataframe and the label encoder models
    dataset_lencoded = dataset.copy(deep=True);
    label_encoders = [None]*dataset.shape[1];

    for i in range(dataset.shape[1]):
        le = LabelEncoder()
        dataset_lencoded.iloc[:,i] = le.fit_transform(dataset_lencoded.iloc[:,i])
        label_encoders[i] = le
    return dataset_lencoded, label_encoders


#Decoder
def label_decoder(dataset_lencoded, label_encoders):

    #Takes in encoded dataframe and list of encoders and returns decoded dataset
    #Label encoder is a list of label encoder models pertaining to each column
    for i in range(dataset_lencoded.shape[1]):
        dataset_lencoded.iloc[:,i] = label_encoders[i].inverse_transform(dataset_lencoded.iloc[:,i])

    return dataset_lencoded #Now it is decoded

In [442]:
dataset,x = label_encoder(dataset)
dataset

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
0,2,1,4,9,21,0,47,16,2,0,10,4,0,2,3
1,2,1,4,9,21,0,47,16,2,0,10,2,0,4,3
2,2,1,4,9,23,53,66,10,2,0,10,2,3,4,2
3,2,1,4,9,2,39,27,16,2,0,10,2,0,4,4
4,2,3,4,9,0,24,41,1,2,4,10,6,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,1,2,4,9,2,12,62,4,5,0,9,5,0,4,17
301,1,2,4,9,2,12,62,4,5,0,9,5,0,4,17
302,1,0,4,2,4,22,5,4,2,3,9,5,4,4,10
303,1,12,4,9,3,51,57,16,2,3,9,5,0,4,7


### Building Models

In [443]:
 import matplotlib.pyplot as plt
 #from kneed import kneeLocator
 from sklearn.cluster import KMeans
 from sklearn.metrics import silhouette_score
 from sklearn.preprocessing import StandardScaler

In [444]:
kmeans = KMeans(
            init="random",
            n_clusters=3,
            n_init=10,
            max_iter=300,
            random_state=42
  )

In [445]:
kmeans.fit(dataset)

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300, n_clusters=3,
       n_init=10, n_jobs=None, precompute_distances='auto', random_state=42,
       tol=0.0001, verbose=0)

In [446]:
dataset

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
0,2,1,4,9,21,0,47,16,2,0,10,4,0,2,3
1,2,1,4,9,21,0,47,16,2,0,10,2,0,4,3
2,2,1,4,9,23,53,66,10,2,0,10,2,3,4,2
3,2,1,4,9,2,39,27,16,2,0,10,2,0,4,4
4,2,3,4,9,0,24,41,1,2,4,10,6,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,1,2,4,9,2,12,62,4,5,0,9,5,0,4,17
301,1,2,4,9,2,12,62,4,5,0,9,5,0,4,17
302,1,0,4,2,4,22,5,4,2,3,9,5,4,4,10
303,1,12,4,9,3,51,57,16,2,3,9,5,0,4,7


In [447]:
kmeans.labels_


array([2, 2, 0, 1, 2, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0,
       1, 1, 0, 0, 1, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2,
       2, 2, 1, 2, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 2, 2, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1, 1, 2,
       2, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 2, 2,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0,
       2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2,

In [449]:
label_decoder(dataset,x)

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
0,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Grant,Generating Revenue,Corporation,7.0
1,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,7.0
2,Merger/Acquisition,Early Stage VC,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,Israel,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Product In Beta Test,Venture Capital-Backed,6.0
3,Merger/Acquisition,Early Stage VC,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
4,Merger/Acquisition,Seed Round,Information Technology,Software,Application Software,"Mobile, Artificial Intelligence & Machine Lear...",Minsk,Belarus,Formerly VC-backed,Startup,Venture Capital,Seed Round,Startup,Venture Capital-Backed,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,20.0
301,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,20.0
302,IPO,Corporate,Information Technology,Computer Hardware,"Computers, Parts and Peripherals","Manufacturing, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Profitable,Venture Capital-Backed,13.0
303,IPO,Series D,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,10.0


In [450]:
dataset[kmeans.labels_==0]

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
2,Merger/Acquisition,Early Stage VC,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,Israel,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Product In Beta Test,Venture Capital-Backed,6.0
5,Merger/Acquisition,Seed Round,Business Products and Services (B2B),Commercial Services,Media and Information Services (B2B),"SaaS, Mobile, Marketing Tech, TMT, CloudTech &...",San Francisco,United States,Formerly VC-backed,Product Development,Venture Capital,Seed Round,Product Development,Venture Capital-Backed,9.0
6,Buyout/LBO,Series H,Information Technology,Software,Business/Productivity Software,"SaaS, Cybersecurity, TMT, CloudTech & DevOps",Minneapolis,United States,Private Equity-Backed,Profitable,"Debt Financed, Private Equity, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,20.0
11,Merger/Acquisition,Early Stage VC,Information Technology,Software,Database Software,"SaaS, Mobile, Big Data, TMT, CloudTech & DevOps",Seattle,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
12,Merger/Acquisition,Later Stage VC,Information Technology,Software,Business/Productivity Software,"SaaS, Mobile, TMT, CloudTech & DevOps",Seattle,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Seed Round,Generating Revenue,Venture Capital-Backed,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,IPO,Later Stage VC,Information Technology,Software,Entertainment Software,"Virtual Reality, Augmented Reality, Gaming, TM...",San Francisco,United States,Formerly VC-backed,Generating Revenue/Not Profitable,"M&A, Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,17.0
295,IPO,Later Stage VC,Information Technology,Software,Entertainment Software,"Virtual Reality, Augmented Reality, Gaming, TM...",San Francisco,United States,Formerly VC-backed,Generating Revenue/Not Profitable,"M&A, Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,17.0
296,IPO,Later Stage VC,Information Technology,Software,Entertainment Software,"Virtual Reality, Augmented Reality, Gaming, TM...",San Francisco,United States,Formerly VC-backed,Generating Revenue/Not Profitable,"M&A, Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,17.0
303,IPO,Series D,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,10.0


In [451]:
dataset[kmeans.labels_==1]

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
3,Merger/Acquisition,Early Stage VC,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
7,Merger/Acquisition,Early Stage VC,Information Technology,Software,Application Software,"Mobile, LOHAS & Wellness, Big Data, Internet o...",Kitchener,Canada,Formerly VC-backed,Generating Revenue,Venture Capital,Accelerator/Incubator,Generating Revenue,Accelerator/Incubator Backed,6.0
8,Merger/Acquisition,Early Stage VC,Information Technology,Software,Application Software,"Mobile, LOHAS & Wellness, Big Data, Internet o...",Kitchener,Canada,Formerly VC-backed,Generating Revenue,Venture Capital,Accelerator/Incubator,Generating Revenue,Accelerator/Incubator Backed,6.0
9,Merger/Acquisition,Early Stage VC,Information Technology,Software,Application Software,"Mobile, LOHAS & Wellness, Big Data, Internet o...",Kitchener,Canada,Formerly VC-backed,Generating Revenue,Venture Capital,Accelerator/Incubator,Generating Revenue,Accelerator/Incubator Backed,6.0
10,Merger/Acquisition,Early Stage VC,Information Technology,Software,Application Software,"Mobile, LOHAS & Wellness, Big Data, Internet o...",Kitchener,Canada,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,Buyout/LBO,Series E,Information Technology,Software,Software Development Applications,"SaaS, CloudTech & DevOps, Supply Chain Tech",Fulton,United States,Private Equity-Backed,Profitable,"Private Equity, Venture Capital",Later Stage VC,Profitable,Venture Capital-Backed,13.0
287,Buyout/LBO,Series E,Information Technology,Software,Software Development Applications,"SaaS, CloudTech & DevOps, Supply Chain Tech",Fulton,United States,Private Equity-Backed,Profitable,"Private Equity, Venture Capital",Later Stage VC,Profitable,Venture Capital-Backed,13.0
297,Merger/Acquisition,Series F,Information Technology,Software,Communication Software,"SaaS, TMT, CloudTech & DevOps",Hackensack,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,16.0
298,Merger/Acquisition,Series F,Information Technology,Software,Communication Software,"SaaS, TMT, CloudTech & DevOps",Hackensack,United States,Formerly VC-backed,Generating Revenue,"Debt Financed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,16.0


In [452]:
dataset[kmeans.labels_==2]

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
0,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Grant,Generating Revenue,Corporation,7.000000
1,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,7.000000
4,Merger/Acquisition,Seed Round,Information Technology,Software,Application Software,"Mobile, Artificial Intelligence & Machine Lear...",Minsk,Belarus,Formerly VC-backed,Startup,Venture Capital,Seed Round,Startup,Venture Capital-Backed,5.000000
17,Merger/Acquisition,Series B,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Seed Round,Stealth,Venture Capital-Backed,6.000000
18,Merger/Acquisition,Series B,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.000000
19,Merger/Acquisition,Series B,Information Technology,Software,Network Management Software,"Cybersecurity, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,6.000000
27,Merger/Acquisition,Later Stage VC,Information Technology,Software,Software Development Applications,"Mobile, TMT, CloudTech & DevOps",Seattle,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Later Stage VC,Generating Revenue,Venture Capital-Backed,9.976667
28,Merger/Acquisition,Later Stage VC,Information Technology,Software,Software Development Applications,"Mobile, TMT, CloudTech & DevOps",Seattle,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Seed Round,Generating Revenue,Venture Capital-Backed,9.976667
37,Merger/Acquisition,Seed Round,Information Technology,Software,Social/Platform Software,"Mobile, AdTech, TMT, CloudTech & DevOps",London,United Kingdom,Formerly VC-backed,Startup,"Private Equity, Venture Capital",Seed Round,Startup,Venture Capital-Backed,9.000000
47,Merger/Acquisition,Later Stage VC,Information Technology,Software,Communication Software,SaaS,San Jose,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Later Stage VC,Generating Revenue,Venture Capital-Backed,12.000000
